## Libraries

In [163]:
import pandas as pd
import numpy as np
import re
from Modules.validaciones import *
from datetime import datetime, date

---

## Local methods

In [164]:
# Some patterns.

In [165]:
module_pattern = r'\d+|SEMANAL|MENSUAL'
fragments_pattern = r'\s\s--\s|\s\s--'
benef_gp_pattern = r'\s[/]\s'
cleaning_pattern = r'SEMANALES|CLINICO 1|CLINICO 2|SESION SEMANAL|20 MG|40 MG|60 MG|80 MG|CONCENTRADOR DE O2|T6 M3|2 RECARGAS'

In [166]:
# some functions.

In [167]:
# divide a string & concatenate the parts without '/' caracter.

In [168]:
def concat_b_p(string: str):
    xs = re.split(benef_gp_pattern, string)
    concat = xs[0] + xs[1]
    return concat

In [169]:
# clean data according to some patterns.

In [170]:
def clean(string: str):
    xs = re.split(cleaning_pattern, string)
    string2 = ''.join(xs)
    return string2

In [171]:
# add as items into a list the 'CODIGO DE PRACTICA OP', 'Periodo' and 'QPeriodo' values, that are on 'ESTADO DE PRACTICA: APROBADA'.

In [172]:
def parsear(string: str):
    parts = re.findall(module_pattern, string)
    return parts

---

## Input: Base_op_pami.csv
## Output: base_op_pami_filtered.csv

In [173]:
# import & read the 'Base_op_pami' file.

In [174]:
base_op_pami_file = pd.read_csv('Leandro/Archivos/Entrada/Base_op_pami.csv')

In [175]:
codigos_prestador = pd.read_csv('datos/202106/CODIGOS_TXT.csv')

In [176]:
# create a new df with some data of 'base_op_pami_file'.

In [177]:
base_op_pami_file = pd.DataFrame(base_op_pami_file, columns=['NRO. OP', 'NRO. BENEFICIO/GP', 'ESTADO DE PRACTICA: APROBADA'])

In [178]:
# create a list for 'NRO. BENEFICIO' and another for 'GP'

In [179]:
zs = list(map(lambda x: re.split(benef_gp_pattern, x), base_op_pami_file['NRO. BENEFICIO/GP']))
beneficios = list(map(lambda x: x[0], zs))
grados_parentesco = list(map(lambda x: x[1], zs))
beneficios = list(map(lambda x: x.zfill(12), beneficios))

In [180]:
# concatenate 'NRO. BENEFICIO' & 'GP' without the '/' caracter.

In [181]:
base_op_pami_file['NRO. BENEFICIO/GP'] = base_op_pami_file['NRO. BENEFICIO/GP'].apply(concat_b_p)

In [182]:
# add leading zeros to each value.

In [183]:
base_op_pami_file['NRO. BENEFICIO/GP'] = add_zeros_to_left(base_op_pami_file, 'NRO. BENEFICIO/GP', 14)

In [184]:
# insert some columns into 'base_op_pami' df.

In [185]:
base_op_pami_file.insert(2, 'CODIGO DE PRACTICA OP', '')
base_op_pami_file.insert(3, 'Periodo', '')
base_op_pami_file.insert(4, 'QPeriodo', '')

In [186]:
# create: 
# - a list of the description of the modules and submodules.
# - a list of the number of modules and submodules of each OP.
# - a list of the modules and submodules, period and period amount values.

In [187]:
description = []
q_beneficio_list = []
parts = []
tmp_ls = ''
tmp_ls_2 = ''
for i, string in base_op_pami_file['ESTADO DE PRACTICA: APROBADA'].iteritems():
    tmp_ls = re.split(fragments_pattern, string)
    tmp_ls.pop(len(tmp_ls)-1)
    description = description + tmp_ls

    tmp_ls_2 = parsear(clean(string))
    q_beneficio_list.append(len(tmp_ls_2)//3)
    parts = parts + tmp_ls_2

In [188]:
# create:
# - a list of the number of OP according to the number of their modules.
# - a list of the number of BENEFICIO-GP according to the number of their modules.
# - a list of 'NRO. BENEFICIO' full values.
# - a list of 'GP' full values.

In [189]:
op_list = []
benef_gp_list = []
full_beneficios = []
full_grados_parentesco = []
for i, size in enumerate(q_beneficio_list):
    op = base_op_pami_file['NRO. OP'][i]
    benef_gp = base_op_pami_file['NRO. BENEFICIO/GP'][i]
    benef = beneficios[i]
    gp = grados_parentesco[i]
    for j in range(size):
        op_list.append(op)
        benef_gp_list.append(benef_gp)
        full_beneficios.append(benef)
        full_grados_parentesco.append(gp)

In [190]:
# create different lists by the modules or submodules, period & period amount

In [191]:
modules = parts[::3]
period = parts[1::3]
amount = parts[2::3]

In [192]:
# create a list for the days amount of practices.

In [193]:
dias = []
for i, p in enumerate(period):
    if (p == 'MENSUAL'):
        qdias = 30 * int(amount[i])
        dias.append(qdias)
    elif (p == 'SEMANAL'):
        qdias = 7 * int(amount[i])
        dias.append(qdias)
    else:
        dias.append('El periodo ingresado no es válido')

In [194]:
# create a new df with filtered data

In [195]:
base_op_pami_filtered = pd.DataFrame({'NRO. OP': op_list, 'NRO. BENEFICIO-GP': benef_gp_list, 'BENEFICIO': full_beneficios, 
                                      'GP': full_grados_parentesco, 'CODIGO DE PRACTICA OP': modules, 'PERIODO': period, 
                                      'QPERIODO': amount, 'QDIAS': dias, 'ESTADO DE PRACTICA: APROBADA': description})

In [196]:

base_op_pami_filtered

,NRO. OP,NRO. BENEFICIO-GP,BENEFICIO,GP,CODIGO DE PRACTICA OP,PERIODO,QPERIODO,QDIAS,ESTADO DE PRACTICA: APROBADA
0,9919061504,10507300100000,105073001000,00,211001,MENSUAL,3,90,211001 - MODULO MENSUAL CLINICO 1 (X 3)
1,9919061504,10507300100000,105073001000,00,227001,MENSUAL,3,90,227001 - SUBMODULO MENSUAL DE CUIDADOR DE CUAT...
2,9919061504,10507300100000,105073001000,00,219004,MENSUAL,3,90,219004 - SUBMODULO MENSUAL DE KINESIOLOGIA MOT...
3,9919061504,10507300100000,105073001000,00,223002,SEMANAL,3,21,223002 - SUBMODULO SEMANAL DE ENFERMERIA DE DO...
4,9919061166,04009151050500,040091510505,00,211001,MENSUAL,3,90,211001 - MODULO MENSUAL CLINICO 1 (X 3)
...,...,...,...,...,...,...,...,...,...
1177,9917896083,15090099430701,150900994307,01,215001,MENSUAL,3,90,215001 - SUBMODULO MENSUAL DE EQUIPAMIENTO - C...
1178,9917896083,15090099430701,150900994307,01,215002,MENSUAL,3,90,215002 - SUBMODULO MENSUAL DE EQUIPAMIENTO - C...
1179,9917896083,15090099430701,150900994307,01,215003,MENSUAL,3,90,215003 - SUBMODULO MENSUAL DE EQUIPAMIENTO - S...
1180,9917896083,15090099430701,150900994307,01,220002,MENSUAL,3,90,220002 - SUBMODULO MENSUAL DE FONOAUDIOLOGIA D...


# Agrego columna visita (Sabrina)

In [197]:
#agrego columnas
base_op_pami_filtered[['Fecha Fin','Visita','Modulo']] = ''

In [198]:
base_op_pami_filtered.columns

Index(['NRO. OP', 'NRO. BENEFICIO-GP', 'BENEFICIO', 'GP',
       'CODIGO DE PRACTICA OP', 'PERIODO', 'QPERIODO', 'QDIAS',
       'ESTADO DE PRACTICA: APROBADA', 'Fecha Fin', 'Visita', 'Modulo'],
      dtype='object')

In [199]:
#Reordeno las columnas 
base_op_pami_filtered = base_op_pami_filtered[['NRO. OP', 'NRO. BENEFICIO-GP', 'BENEFICIO', 'GP',
       'CODIGO DE PRACTICA OP', 'PERIODO', 'QPERIODO', 'QDIAS', 'Fecha Fin', 'Visita', 'Modulo', 'ESTADO DE PRACTICA: APROBADA'  ]].copy()

In [200]:
#Paso a int para poder comparar
base_op_pami_filtered['CODIGO DE PRACTICA OP'] = base_op_pami_filtered['CODIGO DE PRACTICA OP'].astype(int)

In [201]:
#renombro los datos nulos de la tabla de codigos en el campo tipo servicio prestador 
codigos_prestador['Tipo servicio prestador'] = codigos_prestador['Tipo servicio prestador'].fillna('nulos')

In [202]:
codigos_prestador

,CÓDIGO DE MÓDULO,DESCRIPCIÓN DE MÓDULO,CODIGO DE PRACTICA OP,DESCRIPCIÓN DE PRÁCTICA OP,CODIGO DE PRACTICA TXT,DESCRIPCIÓN DE PRÁCTICA TXT,Tipo servicio prestador,Periodicidad,QxPeriodo,Max_Mes,MODULO
0,211,INTERNACION DOMICILIARIA INTEGRAL - MODULO CLI...,211001.0,MODULO MENSUAL CLINICO 1,211001,MODULO MENSUAL CLINICO 1,nulos,Diario,1.0,30.0,SI
1,211,INTERNACION DOMICILIARIA INTEGRAL - MODULO CLI...,211001.0,MODULO MENSUAL CLINICO 1,229101,SUPERVISION MEDICA,Médica/o,Quincenal,1.0,2.0,SI
2,211,INTERNACION DOMICILIARIA INTEGRAL - MODULO CLI...,211001.0,MODULO MENSUAL CLINICO 1,223101,SESION DE ENFERMERIA,Enfermera/o,Semanal,2.0,8.0,SI
3,212,INTERNACION DOMICILIARIA INTEGRAL - MODULO CLI...,212001.0,MODULO MENSUAL CLINICO 2,212001,MODULO MENSUAL CLINICO 2,nulos,Diario,1.0,30.0,SI
4,212,INTERNACION DOMICILIARIA INTEGRAL - MODULO CLI...,212001.0,MODULO MENSUAL CLINICO 2,229101,SUPERVISION MEDICA,Médica/o,Semanal,1.0,4.0,SI
...,...,...,...,...,...,...,...,...,...,...,...
74,232,INTERNACION DOMICILIARIA INTEGRAL - MEDICAMENTOS,232040.0,INTERNACION DOMICILIARIA INTEGRAL - MEDICAMENT...,232040,INTERNACION DOMICILIARIA INTEGRAL - MEDICAMENT...,nulos,NaN,NaN,NaN,NO
75,232,INTERNACION DOMICILIARIA INTEGRAL - MEDICAMENTOS,232050.0,INTERNACION DOMICILIARIA INTEGRAL - MEDICAMENT...,232050,INTERNACION DOMICILIARIA INTEGRAL - MEDICAMENT...,nulos,NaN,NaN,NaN,NO
76,232,INTERNACION DOMICILIARIA INTEGRAL - MEDICAMENTOS,232056.0,INTERNACION DOMICILIARIA INTEGRAL - MEDICAMENT...,232056,INTERNACION DOMICILIARIA INTEGRAL - MEDICAMENT...,nulos,NaN,NaN,NaN,NO
77,232,INTERNACION DOMICILIARIA INTEGRAL - MEDICAMENTOS,232057.0,INTERNACION DOMICILIARIA INTEGRAL - MEDICAMENT...,232057,INTERNACION DOMICILIARIA INTEGRAL - MEDICAMENT...,nulos,NaN,NaN,NaN,NO


Funcion para generar visita

In [203]:
def validar_visita(prestador):
    #filtrar 
    df = codigos_prestador[codigos_prestador['CODIGO DE PRACTICA TXT'] == prestador]
    #print(df)
    var = df['Tipo servicio prestador'].min()
    #print(var)
    
    if var == 'nulos':
        return 'NO'
    else:
        return 'si'

In [204]:
base_op_pami_filtered['Visita'] = base_op_pami_filtered['CODIGO DE PRACTICA OP'].apply(validar_visita)

In [205]:
Detalle_op = base_op_pami_filtered.copy()

# Agrego columna Modulo si(modulo) no (submodulo o internacion)

In [214]:
codigos_prestador[['MODULO', 'CODIGO DE PRACTICA TXT']]

,MODULO,CODIGO DE PRACTICA TXT
0,SI,211001
1,SI,229101
2,SI,223101
3,SI,212001
4,SI,229101
...,...,...
74,NO,232040
75,NO,232050
76,NO,232056
77,NO,232057


In [231]:
def modulo(codigo_txt):
    print(codigos_prestador['MODULO'] == 'SI')
    
    if [codigos_prestador['MODULO'] == 'SI']:
        return 'SI'
    else:
        return 'NO'
    

In [232]:
modulo(232056)

0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool


'SI'

In [224]:
#Aplico la funcion
Detalle_op['Modulo'] = Detalle_op['CODIGO DE PRACTICA OP'].apply(modulo)

0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Leng

0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Leng

0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Leng

0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Leng

Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77  

0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Leng

0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Leng

0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Leng

0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Length: 79, dtype: bool
0      True
1      True
2      True
3      True
4      True
      ...  
74    False
75    False
76    False
77    False
78    False
Name: MODULO, Leng

In [162]:
Detalle_op.tail()

,NRO. OP,NRO. BENEFICIO-GP,BENEFICIO,GP,CODIGO DE PRACTICA OP,PERIODO,QPERIODO,QDIAS,Fecha Fin,Visita,Modulo,ESTADO DE PRACTICA: APROBADA
1177,9917896083,15090099430701,150900994307,01,215001,MENSUAL,3,90,,si,SI,215001 - SUBMODULO MENSUAL DE EQUIPAMIENTO - C...
1178,9917896083,15090099430701,150900994307,01,215002,MENSUAL,3,90,,si,SI,215002 - SUBMODULO MENSUAL DE EQUIPAMIENTO - C...
1179,9917896083,15090099430701,150900994307,01,215003,MENSUAL,3,90,,si,SI,215003 - SUBMODULO MENSUAL DE EQUIPAMIENTO - S...
1180,9917896083,15090099430701,150900994307,01,220002,MENSUAL,3,90,,si,SI,220002 - SUBMODULO MENSUAL DE FONOAUDIOLOGIA D...
1181,9917896083,15090099430701,150900994307,01,221002,MENSUAL,3,90,,si,SI,221002 - SUBMODULO MENSUAL DE TERAPIA OCUPACIO...


In [157]:
base_op_pami_filtered.to_csv('Cabecera_op.csv', index=False, encoding='utf-8')